In [1]:
import warnings
warnings.filterwarnings('ignore')

!pip install -qU langchain langchain-community langchain-huggingface sentence-transformers faiss-cpu pypdf langchain-text-splitters

from google.colab import files
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 24.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently t

In [2]:
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving 1009.1622v1.pdf to 1009.1622v1.pdf


In [3]:
docs = PyPDFLoader(pdf_path).load()
print(f"Loaded {len(docs)} pages from {pdf_path}")

Loaded 110 pages from 1009.1622v1.pdf


In [4]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
chunks = splitter.split_documents(docs)
print(f"Split into {len(chunks)} chunks")

Split into 243 chunks


In [5]:
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vs = FAISS.from_documents(chunks, embedding=emb)
retriever = vs.as_retriever(search_kwargs={"k": 5})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
query = "What physical processes does MESA include to model stellar interiors"
hits = retriever.invoke(query)

In [7]:
print("\n=== Top Results (Naive RAG) ===")
context_texts = []
for i, h in enumerate(hits, 1):
    print(f"\n[{i}] p.{h.metadata.get('page', 'NA')} — {h.page_content[:600]}...")
    context_texts.append(h.page_content)


=== Top Results (Naive RAG) ===

[1] p.98 — dimensional, such as convection, rotation, and magnetism. Those applications remain in the
realm of research frontiers with evolving understanding and insights, quite often profound.
However, much remains to be gained scientiﬁcally (and pedagogically) by accurate one-
dimensional calculations, and this is the present focus of MESA.
A.2. MESA philosophy
The MESA code library project is open. It explicitly invites participation from anybody
(researchers, students, interested amateurs). Participation in MESA can take a wide range of
forms, from just using a MESA release for a science project, to ...

[2] p.5 — Schwarzschild (1958), Cox & Giuli (1968), Clayton (1984), Iben (1991), Hansen & Kawaler
(1995), Arnett (1996), and Kippenhahn & Weigert (1996).
The MESA codebase is in constant development, and future capabilities and applications
will be detailed in subsequent papers. The paper is outlined as follows: §2 explains the
design and implement

In [8]:
context = "\n\n".join(context_texts)

In [9]:
qa_prompt = f"Answer the following question based on the provided context.\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"
llm = pipeline("text2text-generation", model="google/flan-t5-base")
answer = llm(qa_prompt, max_length=512, do_sample=False)[0]["generated_text"]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (1107 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [10]:
print("\n=== Final Answer ===")
print(answer)


=== Final Answer ===
systematic study of the effect of mesh and time-step refinement on simulation accuracy ( 6.7), specific module comparisons (5.5), and stellar evolution code comparisons presented in this section
